<a href="https://colab.research.google.com/github/Xiezhihaa/QOSF-Screening-Task/blob/main/Task%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

QOSF Task2 Odd to Even

In [ ]:
!pip install qiskit
!pip install qiskit[visualization]
!pip install qiskit_aer

In [9]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

Classical computer

In [10]:
def odd_to_even(n,numbers):
    converted = []
    for num in numbers:
        if num%2 != 0:
            num += 1
        if num >= n:
            a = 0
            a = num - n
            num = num - a
            if num%2 != 0:
                num -= 1
        converted.append(num)
    return converted



input_list = [1,2,2,4,5,6,7,11,17,21,22,23]
n = 31
print(odd_to_even(n,input_list))

[2, 2, 2, 4, 6, 6, 8, 12, 18, 22, 22, 24]


Quantum Computer

In [11]:
def initialize_qubits(circuit, qubits, number):
    """Initialize the qubits to a specific number in binary representation."""
    binary = bin(number)[2:].zfill(len(qubits))
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            circuit.x(qubits[i])

In [12]:
def odd_to_even_qc(numbers, n):
    # Quantum circuit to convert odd numbers to even
    results = []

    # Number of qubits
    n_qubits = len(bin(max(numbers)).replace("0b", ""))

    for number in numbers:
        qr = QuantumRegister(n_qubits)
        cr = ClassicalRegister(n_qubits)
        qc = QuantumCircuit(qr, cr)

        # Initialize the qubits to represent 'number'
        initialize_qubits(qc, qr, number)

        # Apply QFT to use phase encoding
        qc.append(QFT(n_qubits), qr)

        # Conditional phase gate if LSB is 1 (indicating the number is odd)
        qc.cp(np.pi, qr[0], qr[1])

        # Apply inverse QFT
        qc.append(QFT(n_qubits).inverse(), qr)

        # Measure the qubits
        qc.measure(qr, cr)

        # Execute the circuit
        simulator = AerSimulator()
        circ = transpile(qc, simulator)
        result = simulator.run(circ).result()

        measured = int(list(result.get_counts(qc).keys())[0], 2)
        if measured >= n or measured < 1:  # Check if the result exceeds the allowed range
            measured -= 2  # Correct by subtracting 2 if it exceeds
        elif measured%2 != 0:
            measured +=1
        results.append(measured)

    return results

In [14]:
output_list = odd_to_even_qc(input_list, n)
print("Output List:", output_list)

Output List: [26, 18, 2, 20, 6, 14, 8, 20, 2, 6, 6, 8]
